In [ ]:
import urllib
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
from PIL import Image, ImageFont, ImageDraw
from IPython import display
import time
from io import BytesIO
import ipywidgets as widgets
import cv2
print("Opencv version: "+cv2.__version__)
import torch
print("Torch version: "+torch.__version__)
#import torchvision
#print("Torchvision version: "+torchvision.__version__)

In [ ]:
url, filename = ("https://upload.wikimedia.org/wikipedia/commons/f/f6/View_of_Valdivia_from_Pedro_de_Valdivia_bridge.jpg", "asd.jpg")
urllib.request.urlretrieve(url, filename)
img = Image.open('asd.jpg')
!rm asd.jpg
display.display(img)

# Procesamiento de imágenes digitales

Una imagen está compuesta de píxeles

Cada píxel nos da información de color en una cierto lugar de la imagen

El color está codificado como tres números: RGB



In [ ]:
from matplotlib.colors import LinearSegmentedColormap
data = np.array(img)[200:300, 50:250, :]
fig, ax = plt.subplots(1, 3, figsize=(9, 2), tight_layout=True)
for k in range(3):
    b = np.zeros((3, ))
    b[k] = 1
    custom_cmap = LinearSegmentedColormap.from_list('custom_cmap', [(0, 0, 0), b], N=100)
    ax[k].imshow(data[:, :, k], cmap=custom_cmap)
    ax[k].axis('off')

In [ ]:
img_bw = 0.2989*data[:, :, 0] + 0.587*data[:, :, 1]+ 0.114*data[:, :, 2]

fig, ax = plt.subplots(1, figsize=(9, 2), tight_layout=True)
ax.imshow(img_bw, cmap=plt.cm.Greys_r)
ax.axis('off');

Los computadores ven las imagenes como arreglos de números

In [ ]:
img_bw 

Podemos operar estos arreglos

In [ ]:
D = 3
filt = np.zeros(shape=(D, D))
filt[1:-1, 1:-1] = 1
display(filt)
img_res = scipy.signal.correlate2d(data.astype('float32'), filt/np.sum(filt), mode='valid')

fig, ax = plt.subplots(1, 3, figsize=(8, 3), tight_layout=True)
ax[0].imshow(subimg, cmap=plt.cm.Greys_r)
ax[1].imshow(filt, cmap=plt.cm.Greys_r)
ax[2].imshow(img_res, cmap=plt.cm.Greys_r);

# Localización y clasificación de entidades en una imagen

Descargamos una imagen "cualquiera" de internet

In [ ]:
url, filename = ("http://4.bp.blogspot.com/_PED_9yjYLVs/S6-fsdvtV9I/AAAAAAAAAGY/3py429MtnQ4/s1600/perros+vagos.jpg", "dogs.jpg")
urllib.request.urlretrieve(url, filename)
img = Image.open('dogs.jpg')
!rm dogs.jpg
display.display(img)

Usaremos un modelo Faster RCNN implementado en pytorch y entrenado en el dataset [COCO](https://github.com/nightrome/cocostuff/blob/master/labels.md)

In [ ]:
label2name = {1: 'persona', 2: 'bicicleta', 3: 'auto', 4: 'moto', 
              8: 'camioneta', 18: 'perro'}

from torchvision.models.detection import fasterrcnn_resnet50_fpn
# Descargo un modelo detector pre-entrenado
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval();
model = model.to('cuda:0')

El modelo retorna 
- las coordenadas de la detección (bounding box)
- la etiqueta (label) de la detección
- la probabilidad de la detección 

In [ ]:
transform = torchvision.transforms.ToTensor()
img_tensor = transform(img)
img_tensor = img_tensor.to('cuda:0')
result = model(img_tensor.unsqueeze(0))[0]

def filter_results(result, threshold=0.9):
    mask = result['scores'] > 0.9
    bbox = result['boxes'][mask].detach().cpu().numpy()
    lbls = result['labels'][mask].detach().cpu().numpy()
    return bbox, lbls

In [ ]:
fnt = ImageFont.truetype("arial.ttf", 30) 

def draw_rectangles(img, bbox, lbls):
    draw = ImageDraw.Draw(img)
    for k in range(len(bbox)):
        if lbls[k] in label2name.keys():
            draw.rectangle(bbox[k], fill=None, outline='white', width=4)
            draw.text([int(d) for d in bbox[k][:2]], label2name[lbls[k]], font=fnt, fill='white')

bbox, lbls = filter_results(result)
draw_rectangles(img, bbox, lbls)
display.display(img)

# Localización y clasificación de entidades en un video

Podemos realizar el mismo procedimiento en cada uno de los cuadros de un video

In [ ]:
fnt = ImageFont.truetype("arial.ttf", 40) 
out = widgets.Output(layout=widgets.Layout(height='480px', width = '720px', border='none'))
display.display(out)            
vid = cv2.VideoCapture('valdivia.mp4')

try:
    while True:
        for k in range(2): # drop one frame
            ret, frame = vid.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        torch_frame = transform(frame)
        torch_frame = torch_frame.to('cuda:0')
        result = model(torch_frame.unsqueeze(0))[0]
        bbox, lbls = filter_results(result, threshold=0.1)
        img = Image.fromarray(frame)
        draw_rectangles(img, bbox, lbls)
        with out:       
            buffer = BytesIO() 
            img.save(buffer, format='JPEG')
            display.display(display.Image(data=buffer.getvalue()))
            display.clear_output(wait=True)
        #time.sleep(0.1)
        
except KeyboardInterrupt:
    vid.release()
#https://github.com/NicksonYap/Jupyter-Webcam/blob/master/Realtime_video_ipython_py3.ipynb

# [Redes Neuronales](https://docs.google.com/presentation/d/16lErEZ2VBt8a-K4lrgdmvaaTOma5sDXvW5fUbLS6qWE/edit#slide=id.g33ee194b7d_0_889)

# http://thispersondoesnotexist.com